In [1]:
import torch

In [2]:
from torch.utils.data import Dataset
from torchvision import transforms
import os
import PIL.Image as Image
import json, random
import numpy as np
import torch
import glob
class Weight_Matrix_Dataset(Dataset):
    def __init__(self, dataset, dataset_stats):
        # split = 'train' or 'val'

        self.dataset = dataset
        
        self.mean = dataset_stats['mean_channel']
        self.std = dataset_stats['std_channel']
        

    def __len__(self): 
        return len(self.dataset)
    
    def normalize_tensor(self, X):
        X = (X - self.mean) / self.std
        return X.requires_grad_(False)

    def __getitem__(self, idx): 
        
        img = self.dataset[idx]
        img = self.normalize_tensor(img)
        
        return img

def get_datasets(dataset_folder_path):
    data = torch.load(dataset_folder_path)
    
    # 질문. dataset_stats은 train 정보만 있는건지? 
    dataset_stats = torch.load(dataset_folder_path.replace('dataset.pt', 'dataset_stats.pt'))
    
    train_dataset = Weight_Matrix_Dataset(data['train'], dataset_stats['train'])
    valid_dataset = Weight_Matrix_Dataset(data['val'], dataset_stats['val'])  
    
    return train_dataset, valid_dataset

In [3]:
train_dataset, valid_dataset = get_datasets('/home/jgryu/Weight_compression/Wparam_dataset/dataset_2d/models--meta-llama--Meta-Llama-3-8B/mlp_attn__512_512_dataset.pt')

In [4]:
from models import Learnable_SEEDLM

/home/jgryu/miniconda3/envs/Wcomp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = Learnable_SEEDLM(input_dim=512, u_length = 4)

In [6]:
x = train_dataset[0].unsqueeze(0).repeat(4,1,1)

In [7]:
# LIC-TCM에서 (4, 256, 256)을 넣으니 (192, 1, 1),  (4, 192, 4, 4)가 나옴
# 그러면... (4, 512, 4)를 넣으면 (512, 1) 가 나와야겠군
model(x)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1536x1 and 512x16)